In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from balance_dataset import balance_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [29]:
df = pd.read_csv('Final/dataset_2.csv')
# add the target column to the features

df['gender'] = 0
df['inpatient'] = 0
df['age'] = 0
df['mean'] = 0
df['std'] = 0
df['max'] = 0
df['min'] = 0
df['target'] = 0

denoised_data = pd.read_csv('Data/denoised_action.csv')
handcraft_data = pd.read_csv("Data/handcraft_data.csv")
tabular = pd.read_csv("Data/scores_for classification_2.csv")
dataset = pd.read_csv("Final/dataset_2.csv")

# assign the first 359 patients (17232/48)
for i in range(0, 359):
    real_patient = denoised_data.loc[i]["patient"]
    real_patient_pos = real_patient -1
    t = tabular.loc[real_patient_pos]["afftype"]
    df.loc[i, 'gender'] = tabular.loc[real_patient_pos]['gender']
    df.loc[i, 'inpatient'] = tabular.loc[real_patient_pos]['inpatient']
    df.loc[i, 'age'] = tabular.loc[real_patient_pos]['age']
    df.loc[i, 'mean'] = handcraft_data.loc[real_patient_pos]['means']
    df.loc[i, 'std'] = handcraft_data.loc[real_patient_pos]['stds']
    df.loc[i, 'max'] = handcraft_data.loc[real_patient_pos]['max']
    df.loc[i, 'min'] = handcraft_data.loc[real_patient_pos]['min']
    if int(t) == 1 or int(t) == 3:
        df.loc[i, 'target'] = 0
    else:
        df.loc[i, 'target'] = 1

In [30]:
df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f29,f30,gender,inpatient,age,mean,std,max,min,target
0,0.843199,-0.019875,0.348144,-3.734176,0.793592,-2.311102,0.872043,-0.625631,0.684587,-0.424186,...,1.505166,-2.901132,2,2,4,156.247222,158.294357,478.600000,0.0,1
1,2.384476,0.075164,-1.233456,-1.727925,-0.328627,-1.431436,1.717354,-1.287621,2.351772,-1.477809,...,0.746082,-2.343760,2,2,4,156.247222,158.294357,478.600000,0.0,1
2,0.332048,-1.513692,1.440681,-2.308658,-0.041846,-1.517303,-0.487194,-0.846304,0.269623,-2.833039,...,2.174545,-2.293868,2,2,4,156.247222,158.294357,478.600000,0.0,1
3,-1.402971,-0.658496,0.481852,-3.419377,0.539492,-1.853214,1.294299,-1.714455,1.201168,-1.017584,...,2.777213,-2.310588,2,2,4,156.247222,158.294357,478.600000,0.0,1
4,0.852535,0.862926,2.134740,-2.638090,-0.442957,-0.718977,0.090587,0.103345,0.841696,-1.218732,...,1.594197,-1.063941,2,2,4,156.247222,158.294357,478.600000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,-1.600638,0.283824,2.018573,-1.191005,-0.657852,-2.721894,-0.059204,-0.194050,-0.025236,-1.738701,...,-0.503800,0.401390,1,1,3,338.096528,504.973433,2186.633333,4.0,1
355,-0.725218,0.192610,0.117516,-1.099034,-1.249787,-0.897582,0.777956,-0.244736,0.278399,-1.449569,...,-0.256283,-0.925431,1,1,3,338.096528,504.973433,2186.633333,4.0,1
356,-0.403742,-0.144096,0.103674,-0.749376,-1.266680,-1.128065,0.482980,-0.848260,-0.348291,-0.108733,...,-0.539237,-0.754682,1,1,3,338.096528,504.973433,2186.633333,4.0,1
357,0.790339,-0.491060,-0.851775,-1.025607,-1.013319,-0.679107,0.924477,-0.230366,0.313328,-0.793535,...,0.183362,-0.272740,1,1,3,338.096528,504.973433,2186.633333,4.0,1


In [31]:
counts = df['target'].value_counts()
print(f'Before balancing :{counts}')
# balance the dataset
dataset = balance_dataset(df)
counts = dataset['target'].value_counts()
print(f'After: {counts}')

# scale the data from all the features except the target
scaler = StandardScaler()
dataset.iloc[:, 1:-1] = scaler.fit_transform(dataset.iloc[:, 1:-1])

# split to features and targets
y = dataset['target']
X = dataset.loc[:, ~dataset.columns.isin(['patient', 'target'])]

Before balancing :1    233
0    126
Name: target, dtype: int64
After: 0    126
1    126
Name: target, dtype: int64


In [33]:
# split the data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print()
print(f'We have {len(X_train)} patients in the trainig set')
print(f'and {len(X_test)} patients in the test set')
print()
print('-----------------------------')

# convert the data to tensors
X_train_ = torch.tensor(X_train.values, dtype=torch.float32)
X_test_ = torch.tensor(X_test.values, dtype=torch.float32)
y_train_ = torch.tensor(y_train.values, dtype=torch.float32)
y_test_ = torch.tensor(y_test.values, dtype=torch.float32)


We have 201 patients in the trainig set
and 51 patients in the test set

-----------------------------


In [34]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(37, 15)
        self.fc2 = nn.Linear(15, 7)
        self.fc3 = nn.Linear(7, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # we have classification problem so we will use sigmoid function
        x = torch.sigmoid(self.fc3(x))
        return x

# set the seeds
torch.manual_seed(42)
np.random.seed(42)

In [38]:
# create the loss function and the optimizer. We will use MSE loss function
criterion = nn.BCELoss() # Binary Cross Entropy loss for binary classification
#optimizer = torch.optim.Adam(net.parameters(), lr=0.1, weight_decay=0)
# another optimizer that we can use is SGD

# run model_1.py for different hyperparameters
# and save the results in a csv file
# lr = learning rate
# wd = weight decay
# mm = momentum
# ld = learning rate decay

for lr in [0.01, 0.001]:
    for wd in [ 0.001]:
        for mm in [0.5, 0.725]:
            for ld in [1, 0.9]:
                hyperparameters = {'lr': lr, 'weight_decay': wd, 'momentum': mm}
                lr_decay = ld
                print(f'lr = {lr}, wd = {wd}, mm = {mm}, ld = {ld}')
                net = Net()


                hyperparameters = {'lr': lr, 'weight_decay': wd, 'momentum': mm}
                lr_decay = ld

                optimizer = torch.optim.SGD(net.parameters(), **hyperparameters)


                # train the model
                epochs = 5000
                loss_values = []

                from sklearn.model_selection import KFold

                # Define the number of folds for cross-validation
                num_folds = 5

                # Create a KFold object
                kf = KFold(n_splits=num_folds, shuffle=True)
                for fold, (train_ids, val_ids) in enumerate(kf.split(X_train_)):
                    X_train_fold = X_train_[train_ids]
                    y_train_fold = y_train_[train_ids]
                    X_val_fold = X_train_[val_ids]
                    y_val_fold = y_train_[val_ids]


                    for i in range(epochs):
                        i += 1
                        y_pred = net.forward(X_train_fold)

                        loss = criterion(y_pred, y_train_fold.unsqueeze(1).float())
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        # print every 10 epochs
                        if i%1000 == 0:
                            print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
                            loss_values.append([i, loss.item()])
                        # decrease the learning rate every 300 epochs
                        if i % 1000 == 0:
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] * lr_decay



                # test the model
                with torch.no_grad():
                    y_val = net.forward(X_test_)
                    loss = criterion(y_val, y_test_.unsqueeze(1).float())

                # calculate the accuracy
                correct = 0
                total = 0
                correct_predictions = []
                prediction = []
                with torch.no_grad():
                    predictions = net.forward(X_test_)
                    for i in range(len(y_test_)):
                        if predictions[i] >= 0.5:
                            y_pred = 1
                            prediction.append(1)
                        else:
                            y_pred = 0
                            prediction.append(0)
                        if y_pred == y_test_[i]:
                            correct += 1
                            print(f'correct prediction: {i}')
                            correct_predictions.append(i)
                        total += 1
                print()
                accuracy = correct/total
                print(f'loss on the test set = {loss:.3f}')
                print(f'Accuracy on the test set: {round(accuracy, 4)*100:3.4f}%')



                # csv with colums: epochs, lr, weight_decay, momentum, accuracy
                # in order to find the best hyperparameters
                df = pd.read_csv('NN_hyperparameters.csv')



                df = pd.concat([df, pd.DataFrame([[epochs, hyperparameters['lr'], hyperparameters['weight_decay'], \
                                                    hyperparameters['momentum'], lr_decay, accuracy]], columns=['epochs', 'lr', \
                                                    'weight_decay', 'momentum', 'lr_decay',  'accuracy'])], axis=0, ignore_index=True)

                # change the order of the columns
                df = df[['epochs', 'lr', 'weight_decay', 'momentum', 'lr_decay',  'accuracy']]

                df.to_csv('NN_hyperparameters.csv', index=False, header=True)


                # save the model's weights in order to plot the features with their weights
                torch.save(net.state_dict(), 'NN_weights.pt')


                # merge X_test with y_test and save it to csv
                X_test_df = pd.DataFrame(X_test_)
                y_test_df = pd.DataFrame(y_test_)
                predictions_df = pd.DataFrame(prediction)
                # merge X_test_df with y_test_df
                test_df = pd.concat([X_test_df, y_test_df, predictions_df], axis=1, ignore_index=True)

                # header
                header = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', \
                        'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', \
                            'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', \
                                'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'gender', \
                                 	'inpatient', 	'age',	'mean' ,	'std', 	'max', 	'min', 'target', 'prediction']


                print(test_df)
                # save the test_df to csv
                test_df.to_csv('test_df.csv', index=False, header=header)

lr = 0.01, wd = 0.001, mm = 0.5, ld = 1
epoch: 1000  loss: 0.50879633
epoch: 2000  loss: 0.19837317
epoch: 3000  loss: 0.09366910
epoch: 4000  loss: 0.06743531
epoch: 5000  loss: 0.05880526
epoch: 1000  loss: 0.09826312
epoch: 2000  loss: 0.09095439
epoch: 3000  loss: 0.08873633
epoch: 4000  loss: 0.08710450
epoch: 5000  loss: 0.08610415
epoch: 1000  loss: 0.10934492
epoch: 2000  loss: 0.10608510
epoch: 3000  loss: 0.10457227
epoch: 4000  loss: 0.10367596
epoch: 5000  loss: 0.10299361
epoch: 1000  loss: 0.09488130
epoch: 2000  loss: 0.09419014
epoch: 3000  loss: 0.09317821
epoch: 4000  loss: 0.09269316
epoch: 5000  loss: 0.09230428
epoch: 1000  loss: 0.10568133
epoch: 2000  loss: 0.10446747
epoch: 3000  loss: 0.10292778
epoch: 4000  loss: 0.10149106
epoch: 5000  loss: 0.09995227
correct prediction: 0
correct prediction: 2
correct prediction: 3
correct prediction: 4
correct prediction: 5
correct prediction: 6
correct prediction: 7
correct prediction: 8
correct prediction: 11
correct pre

In [44]:
# create the loss function and the optimizer. We will use MSE loss function
criterion = nn.BCELoss() # Binary Cross Entropy loss for binary classification
#optimizer = torch.optim.Adam(net.parameters(), lr=0.1, weight_decay=0)
# another optimizer that we can use is SGD

# run model_1.py for different hyperparameters
# and save the results in a csv file
# lr = learning rate
# wd = weight decay
# mm = momentum
# ld = learning rate decay

for lr in [0.01]:
    for wd in [ 0.001]:
        for mm in [0.5]:
            for ld in [1]:
                hyperparameters = {'lr': lr, 'weight_decay': wd, 'momentum': mm}
                lr_decay = ld
                print(f'lr = {lr}, wd = {wd}, mm = {mm}, ld = {ld}')
                net = Net()


                hyperparameters = {'lr': lr, 'weight_decay': wd, 'momentum': mm}
                lr_decay = ld

                optimizer = torch.optim.SGD(net.parameters(), **hyperparameters)


                # train the model
                epochs = 10000
                loss_values = []

                from sklearn.model_selection import KFold

                # Define the number of folds for cross-validation
                num_folds = 5

                # Create a KFold object
                kf = KFold(n_splits=num_folds, shuffle=True)
                for fold, (train_ids, val_ids) in enumerate(kf.split(X_train_)):
                    X_train_fold = X_train_[train_ids]
                    y_train_fold = y_train_[train_ids]
                    X_val_fold = X_train_[val_ids]
                    y_val_fold = y_train_[val_ids]


                    for i in range(epochs):
                        i += 1
                        y_pred = net.forward(X_train_fold)

                        loss = criterion(y_pred, y_train_fold.unsqueeze(1).float())
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        # print every 10 epochs
                        if i%1000 == 0:
                            print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
                            loss_values.append([i, loss.item()])
                        # decrease the learning rate every 300 epochs
                        if i % 1000 == 0:
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] * lr_decay



                # test the model
                with torch.no_grad():
                    y_val = net.forward(X_test_)
                    loss = criterion(y_val, y_test_.unsqueeze(1).float())

                # calculate the accuracy
                correct = 0
                total = 0
                correct_predictions = []
                prediction = []
                with torch.no_grad():
                    predictions = net.forward(X_test_)
                    for i in range(len(y_test_)):
                        if predictions[i] >= 0.5:
                            y_pred = 1
                            prediction.append(1)
                        else:
                            y_pred = 0
                            prediction.append(0)
                        if y_pred == y_test_[i]:
                            correct += 1
                            print(f'correct prediction: {i}')
                            correct_predictions.append(i)
                        total += 1
                print()
                accuracy = correct/total
                print(f'loss on the test set = {loss:.3f}')
                print(f'Accuracy on the test set: {round(accuracy, 4)*100:3.4f}%')



                # csv with colums: epochs, lr, weight_decay, momentum, accuracy
                # in order to find the best hyperparameters
                df = pd.read_csv('NN_hyperparameters.csv')



                df = pd.concat([df, pd.DataFrame([[epochs, hyperparameters['lr'], hyperparameters['weight_decay'], \
                                                    hyperparameters['momentum'], lr_decay, accuracy]], columns=['epochs', 'lr', \
                                                    'weight_decay', 'momentum', 'lr_decay',  'accuracy'])], axis=0, ignore_index=True)

                # change the order of the columns
                df = df[['epochs', 'lr', 'weight_decay', 'momentum', 'lr_decay',  'accuracy']]

                df.to_csv('NN_hyperparameters.csv', index=False, header=True)


                # save the model's weights in order to plot the features with their weights
                torch.save(net.state_dict(), 'NN_weights.pt')


                # merge X_test with y_test and save it to csv
                X_test_df = pd.DataFrame(X_test_)
                y_test_df = pd.DataFrame(y_test_)
                predictions_df = pd.DataFrame(prediction)
                # merge X_test_df with y_test_df
                test_df = pd.concat([X_test_df, y_test_df, predictions_df], axis=1, ignore_index=True)

                # header
                header = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', \
                        'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', \
                            'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', \
                                'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'gender', \
                                 	'inpatient', 	'age',	'mean' ,	'std', 	'max', 	'min', 'target', 'prediction']


                print(test_df)
                # save the test_df to csv
                test_df.to_csv('test_df.csv', index=False, header=header)

lr = 0.01, wd = 0.001, mm = 0.5, ld = 1
epoch: 1000  loss: 0.43767014
epoch: 2000  loss: 0.15775484
epoch: 3000  loss: 0.09176248
epoch: 4000  loss: 0.07946526
epoch: 5000  loss: 0.07593711
epoch: 6000  loss: 0.07191874
epoch: 7000  loss: 0.07066028
epoch: 8000  loss: 0.07000837
epoch: 9000  loss: 0.06961896
epoch: 10000  loss: 0.06936412
epoch: 1000  loss: 0.08714816
epoch: 2000  loss: 0.08552241
epoch: 3000  loss: 0.08347196
epoch: 4000  loss: 0.08294921
epoch: 5000  loss: 0.08271031
epoch: 6000  loss: 0.08257045
epoch: 7000  loss: 0.08248807
epoch: 8000  loss: 0.08242690
epoch: 9000  loss: 0.08239860
epoch: 10000  loss: 0.08237817
epoch: 1000  loss: 0.08628485
epoch: 2000  loss: 0.08525656
epoch: 3000  loss: 0.08482327
epoch: 4000  loss: 0.08457090
epoch: 5000  loss: 0.08440936
epoch: 6000  loss: 0.06684068
epoch: 7000  loss: 0.06588088
epoch: 8000  loss: 0.06575238
epoch: 9000  loss: 0.04299818
epoch: 10000  loss: 0.04236787
epoch: 1000  loss: 0.03481787
epoch: 2000  loss: 0.034061

In [46]:
!pip install dice_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.0 MB/s eta 0:00:00


In [53]:
y_train.values


array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0])

In [66]:
def predict_proba(x):
    predictions = net.forward(x)
    if predictions >= 0.5:
        y_pred = 1
        prediction = 1
    else:
        y_pred = 0
        prediction =0
    return prediction

AttributeError: ignored

In [94]:
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split
import copy

total_train = copy.deepcopy(X_train)
total_train["target"] = y_train.values

d = dice_ml.Data(dataframe=total_train,
                 continuous_features=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', \
                        'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', \
                            'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', \
                                'f25', 'f26', 'f27', 'f28', 'f29', 'f30',\
                                      'mean' ,	'std', 	'max', 	'min'],
                 outcome_name='target')

m = dice_ml.Model(model=net, backend="PYT")

exp = dice_ml.Dice(d, m, method="gradient")

In [79]:
X_test[0:1].to_numpy()

array([[ 0.23579499, -0.59652794,  0.76493944,  0.1551249 , -0.70939181,
        -1.9215456 , -0.11027424,  0.38222368, -1.06499406, -2.58115972,
        -0.38419395,  1.0881695 ,  0.96168222, -1.6384771 , -1.10662867,
        -1.17581983,  0.28103921,  1.63970257, -0.72448319,  0.47746048,
        -0.34333624,  1.28896   ,  2.17539774,  0.57472378,  2.34580742,
         0.70065716, -0.26703981, -0.92457977, -0.26234671,  0.21956124,
        -1.16410489, -2.30217289, -1.03529127,  2.58499983,  2.817139  ,
         2.57130546,  1.73671759]])

In [95]:
# Generate counterfactual examples
query_instance = X_test[0:1]
exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")
# Visualize counterfactual explanation
exp.visualize_as_dataframe()

100%|██████████| 1/1 [11:09<00:00, 669.99s/it]

Diverse Counterfactuals found! total time taken: 00 min 07 sec
Query instance (original outcome : 1)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f29,f30,gender,inpatient,age,mean,std,max,min,target
0,0.235795,-0.596528,0.764939,0.155125,-0.709392,-1.921546,-0.110274,0.382224,-1.064994,-2.58116,...,-0.262347,0.219561,-1.164105,-2.302173,-1.035291,2.585,2.817139,2.571306,1.736718,1.0



Diverse Counterfactual set (new outcome: 0.0)


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f29,f30,gender,inpatient,age,mean,std,max,min,target
0,0.230737,-3.535150e-11,0.617340,0.135508,3.017268e-11,-1.811280e-10,-0.001,0.378796,9.106870e-09,-6.759055e-09,...,-5.896983e-10,0.207510,1.0,1.0,1.0,0.435374,1.0,1.0,1.0,0
1,0.164868,-3.535150e-11,0.762884,0.306157,3.017268e-11,-1.811280e-10,-0.001,0.483216,7.037346e-02,-6.759055e-09,...,-5.896983e-10,0.216372,1.0,1.0,1.0,0.994747,1.0,1.0,1.0,0
2,0.386192,-3.535150e-11,0.892289,0.155406,3.017268e-11,-1.811280e-10,-0.001,0.348003,9.106870e-09,-6.759055e-09,...,-5.896983e-10,0.218201,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,0
3,0.255818,-3.535150e-11,0.764296,0.164220,3.017268e-11,-1.811280e-10,-0.001,0.378449,9.106870e-09,-6.759055e-09,...,-5.896983e-10,0.428304,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,0


In [98]:
y_test[0]

1